In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
PROJECT_ROOT = "/content/drive/MyDrive/AI_RL_Task_Scheduling"

DATA_DIR = f"{PROJECT_ROOT}/data"
SRC_DIR = f"{PROJECT_ROOT}/src"
MODELS_DIR = f"{PROJECT_ROOT}/models"
OUTPUTS_DIR = f"{PROJECT_ROOT}/outputs"

sys.path.append(SRC_DIR)


Mounted at /content/drive


In [2]:
%%writefile /content/drive/MyDrive/AI_RL_Task_Scheduling/src/sim_env.py

import numpy as np
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class Task:
    task_id: int
    arrival_time: int
    processing_time: int
    deadline: int
    remaining_time: int
    waiting_time: int = 0
    start_time: Optional[int] = None
    completion_time: Optional[int] = None

    def is_complete(self):
        return self.remaining_time <= 0


class Scheduler:
    def select_task(self, ready_queue: List[Task], t: int) -> int:
        raise NotImplementedError


class SchedulerEnv:
    def __init__(self, tasks: List[Task], scheduler: Scheduler, max_time: int = 100000):
        self.tasks = sorted(tasks, key=lambda x: x.arrival_time)
        self.scheduler = scheduler
        self.max_time = max_time

        self.time = 0
        self.ready_queue: List[Task] = []
        self.completed_tasks: List[Task] = []
        self.task_ptr = 0

    def step(self):
        while self.task_ptr < len(self.tasks) and self.tasks[self.task_ptr].arrival_time <= self.time:
            self.ready_queue.append(self.tasks[self.task_ptr])
            self.task_ptr += 1

        if self.ready_queue:
            idx = self.scheduler.select_task(self.ready_queue, self.time)
            task = self.ready_queue[idx]

            if task.start_time is None:
                task.start_time = self.time

            task.remaining_time -= 1

            for i, other in enumerate(self.ready_queue):
                if i != idx:
                    other.waiting_time += 1

            if task.is_complete():
                task.completion_time = self.time + 1
                self.completed_tasks.append(task)
                self.ready_queue.pop(idx)

        self.time += 1

    def run(self):
        while (
            self.time < self.max_time and
            (self.task_ptr < len(self.tasks) or len(self.ready_queue) > 0)
        ):
            self.step()

        return self.completed_tasks


def compute_metrics(tasks: List[Task]):
    n = len(tasks)

    waiting_times = np.array([t.waiting_time for t in tasks])
    avg_waiting_time = waiting_times.mean()

    deadline_misses = sum(
        1 for t in tasks if t.completion_time > t.deadline
    )
    deadline_miss_rate = deadline_misses / n

    total_time = max(t.completion_time for t in tasks)
    throughput = n / total_time

    return {
        "avg_waiting_time": avg_waiting_time,
        "deadline_miss_rate": deadline_miss_rate,
        "throughput": throughput
    }


Writing /content/drive/MyDrive/AI_RL_Task_Scheduling/src/sim_env.py


In [3]:
from sim_env import Task, Scheduler, SchedulerEnv, compute_metrics
from config import RR_QUANTUM


In [4]:
class FCFSScheduler(Scheduler):
    def select_task(self, ready_queue, t):
        return min(
            range(len(ready_queue)),
            key=lambda i: (ready_queue[i].arrival_time, ready_queue[i].task_id)
        )


In [5]:
class SJFScheduler(Scheduler):
    def select_task(self, ready_queue, t):
        return min(
            range(len(ready_queue)),
            key=lambda i: (ready_queue[i].remaining_time,
                           ready_queue[i].arrival_time,
                           ready_queue[i].task_id)
        )


In [6]:
class EDFScheduler(Scheduler):
    def select_task(self, ready_queue, t):
        return min(
            range(len(ready_queue)),
            key=lambda i: (ready_queue[i].deadline,
                           ready_queue[i].remaining_time,
                           ready_queue[i].task_id)
        )


In [7]:
class RoundRobinScheduler(Scheduler):
    def __init__(self):
        self.pointer = 0

    def select_task(self, ready_queue, t):
        if self.pointer >= len(ready_queue):
            self.pointer = 0

        idx = self.pointer
        self.pointer = (self.pointer + 1) % len(ready_queue)
        return idx


In [8]:
def make_test_tasks():
    tasks = []
    for i in range(10):
        tasks.append(
            Task(
                task_id=i,
                arrival_time=i // 2,
                processing_time=3 + (i % 3),
                remaining_time=3 + (i % 3),
                deadline=i + 15
            )
        )
    return tasks


In [9]:
import pandas as pd

schedulers = {
    "FCFS": FCFSScheduler(),
    "SJF": SJFScheduler(),
    "EDF": EDFScheduler(),
    "RR": RoundRobinScheduler(),
}

results = []

for name, sched in schedulers.items():
    tasks = make_test_tasks()
    env = SchedulerEnv(tasks, sched)
    completed = env.run()
    metrics = compute_metrics(completed)
    metrics["scheduler"] = name
    results.append(metrics)

df = pd.DataFrame(results)
df


,avg_waiting_time,deadline_miss_rate,throughput,scheduler
0,15.4,0.5,0.25641,FCFS
1,13.3,0.5,0.25641,SJF
2,15.4,0.5,0.25641,EDF
3,25.6,1.0,0.25641,RR


In [10]:
df.to_csv(f"{OUTPUTS_DIR}/baseline_sanity_metrics.csv", index=False)
print("Saved baseline sanity metrics to outputs/")


Saved baseline sanity metrics to outputs/
